In [ ]:
import os
import sys
import glob
import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from comet_ml.api import API, APIExperiment
from comet_ml.query import Tag

if '..' not in sys.path:
    sys.path.append('..')
from deep_utils import *

In [ ]:
api = API(api_key = os.environ['COMET_ML_API_KEY'])
workspace = 'danielelinaro'
project_name = 'inertia'
query = Tag('G1') & Tag('D=2') & Tag('DZA=0.036') & Tag('1D_pipeline')
experiments = api.query(workspace, project_name, query, archived=False)
data = {}
for experiment in experiments:
    data[experiment.id] = {}
    print(f'Downloading data for experiment ID {experiment.id}.')
    data[experiment.id]['metrics'] = experiment.get_metrics()
    data[experiment.id]['tags'] = experiment.get_tags()

In [ ]:
if len(data) == 3:
    rows,cols = 1,3
elif len(data) == 4:
    rows,cols = 2,2
fig,ax = plt.subplots(rows, cols, sharex=True, sharey=True, figsize=(cols*4.5, rows*4), squeeze=False)
for i,experiment_id in enumerate(data):
    r = i // cols
    c = i % cols
    activation_loc = [tag for tag in data[experiment_id]['tags'] if 'ReLU' in tag][0]
    loss = np.array([float(m['metricValue']) for m in data[experiment_id]['metrics'] if m['metricName'] == 'loss'])
    val_loss = np.array([float(m['metricValue']) for m in data[experiment_id]['metrics'] if m['metricName'] == 'val_loss'])
    mape = [float(m['metricValue']) for m in data[experiment_id]['metrics'] if m['metricName'] == 'mape_prediction'][0]
    ax[r,c].semilogy(loss, 'k', lw=1, label='Training loss')
    ax[r,c].semilogy(val_loss, 'r', lw=1, label='Validation loss')
    if i == len(data) - 1:
        ax[r,c].legend(loc='upper right')
    ax[r,c].set_xlabel('Epoch')
    if i == 0:
        ax[r,c].set_ylabel('Loss')
    n_epochs = len(loss)
    ax[r,c].text(n_epochs, 0.15, f'Min loss = {loss.min():.3f}', horizontalalignment='right')
    ax[r,c].text(n_epochs, 0.115, f'Min val_loss = {val_loss.min():.3f}', horizontalalignment='right')
    ax[r,c].text(n_epochs, 0.08, f'MAPE = {mape:.2f}%', horizontalalignment='right')
    ax[r,c].set_title(activation_loc.replace('_', ' '));